# Мастерская "Английские субтитры"

# Подготовка субтитров

---

**Входные данные**

Размеченный список фильмов. Содержит целевой признак – значения уровней CEFR, определенные экспертами.

Файлы с субтитрами к фильмам в формате srt.

---

**Цель**

Добавить в таблицу фильмов поле, содержащее слова из субтитров.

---

**Задачи:**  

- сделать обработку субтитров, удалив из них лишнюю информацию (служебные данные, знаки препинания и т. п.);
- проверить соответствие указанных уровней классификации CEFR;
- разработать новые признаки для проверки их оценить их предполагаемое влияние на качество модели.

---

## Initial

### Imports

In [123]:
import pandas as pd
import os
import warnings

import re
from tqdm import tqdm

### Constants

In [124]:
DATASETS_PATH_LOCAL = 'datasets/'                               # local path to data
DATASETS_PATH_REMOTE = '/datasets/'                             # remote path to data

SUBTITLES_PATH_LOCAL = 'datasets/subtitles/'                    # local path to data (subtitles)
SUBTITLES_PATH_REMOTE = '/datasets/subtitles/'                  # remote path to data (subtitles)

CR = '\n'                                                       # new line char sequence

### Functions

In [125]:
def custom_read_csv(file_name, separator=','):
    """
    reading dataset of .csv format:
       first from local storage;
       if unsuccessful from remote storage.
    """

    path_local = f'{PATH_LOCAL}{file_name}'
    path_remote = f'{PATH_REMOTE}{file_name}'
    
    if os.path.exists(path_local):
        return pd.read_csv(path_local, sep=separator)

    elif os.path.exists(path_remote):
        return pd.read_csv(path_remote, sep=separator)

    else:
        print(f'File "{file_name}" not found at the specified path ')

### Settings

In [126]:
# text styles
class f:
    BOLD = "\033[1m"
    ITALIC = "\033[3m"
    END = "\033[0m"

In [127]:
# Pandas defaults
pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 500
pd.options.display.max_columns = 100
pd.options.display.float_format = '{:.3f}'.format
pd.options.display.colheader_justify = 'left'

In [128]:
# others
warnings.filterwarnings('ignore')

---

## Подготовка слов из субтитров к фильмам

### Функция для чтения субтитров из файла

In [129]:
def read_subtitles(file_name, file_ext='srt', file_encoding='latin-1'):
    """
    Чтение текста из файла file_name.
    Возвращает датафрейм с текстом и статус операции:
        'OK' – файл найден;
        'no subtitles' – файл с субтитрами не найден.
    """

    path_local = f'{SUBTITLES_PATH_LOCAL}{file_name}.{file_ext}'
#     print(f'file {path_local} in process')
    
    if os.path.exists(path_local):
        return pd.read_table(path_local, names=['subs'], encoding=file_encoding), 'OK'

    else:
        print(f'file {f.BOLD}{path_local}{f.END} not found')
        return pd.DataFrame(), 'no subtitles'

### Функция для очистки текста

In [130]:
def clean_text(text):
    '''
    Принятый текст в типа string:
        - переводит в нижний регистр;
        - заменяет на пробел символ перевода строки;
        - заменяет на пробел теги (текст, заключенный в квадратные или фигурные скобки);
        - заменяет на пробел все символы, кроме букв, дефисов и апострофов внутри слов;
        - удаляет все слова из 1-2 букв;
        - заменяет цепочки пробелов на одинарный пробел.
    Возвращает очищенный текст.
    '''
    # приведение значений к нижнему регистру
    text = text.lower()
    
    # удаление перевода строки
    text = text.replace('\n', ' ')
    
    # удаление тегов: частей строк между символами < и >, { и }
    text = re.sub('<.*?>', ' ', text)
    text = re.sub('{.*?}', ' ', text)

    #-------------------------------------------------------------------------------------------------
    # удаление всех символов, кроме букв, пробелов и внутренних дефисов и апострофов
    
    # замена дефисов внутри слов на временный заполнитель (перед удалением небуквенных символов)
    text = re.sub('([a-zA-Z])-([a-zA-Z])', '\\1temporarydefis\\2', text)
    # замена апострофов внутри слов на временный заполнитель (перед удалением небуквенных символов)
    text = re.sub("([a-zA-Z])'([a-zA-Z])", '\\1temporaryapostrof\\2', text)
    # удаление всех символов, кроме букв и пробелов
    text = re.sub('[^a-zA-Z\s]', ' ', text)
    # восстановление внутренних дефисов (замена временного заполнителя на настоящий дефис)
    text = text.replace('temporarydefis', '-')
    # восстановление внутренних апострофов (замена временного заполнителя на настоящий апостроф)
    text = text.replace('temporaryapostrof', "'")
    #-------------------------------------------------------------------------------------------------

    # удаление всех символов, кроме букв и пробелов
#     text = re.sub('[^a-zA-Z\s]', ' ', text)                      # на случай, если дефисы и апострофы тоже нужно удалить
    
    # удаление слов из одной-двух букв (за редким исключением)
#     subtitles = re.sub(r'\s\b\w{1,2}\b\s', ' ', subtitles)       # слово окружено пробелами
    text = re.sub(r'\b\w{1,2}\b', ' ', text)                       # неважно, чем окружено слово
    
    # замена нескольких последовательных пробелов на одинарные
    text = re.sub(' +', ' ', text)
    
    # удаление крайних пробелов (мелочь, но пусть будет)
    text = text.strip()

    return text

### Функция для подготовки субтитров

In [131]:
def prepare_subtitles(movie_name, clean=True):
    '''
    Загружает субтитры для указанного фильма movie_name с помощью функции read_subtitles.
    Выполняет очистку текста от лишней информации (служебные данные, знаки препинания и т.п.) с помощью функции clean_text.
    Возвращает очищенный текст.
    '''
    
    # чтение файла с субтитрами; имя файла совпадает с названием фильма в таблице фильмов
    df, status = read_subtitles(movie_name)
    if status == 'no subtitles':
        return 'no subtitles'
    
    # удаление служебных строк (начинаются с цифры)
    df = df[~df.subs.str.match('^\d')].reset_index(drop=True)
    
    # соединение отдельных субтитров в единый корпус
    subtitles = df.subs.str.cat(sep=' ')
    
    if clean:
        subtitles = clean_text(subtitles)
    
    return subtitles

### Чтение таблицы с фильмами

In [132]:
data = pd.read_excel(DATASETS_PATH_LOCAL + 'movies_labels.xlsx', index_col='id').reset_index(drop=True)

In [133]:
data.sample(3)

,Movie,Level
102,The_usual_suspects(1995),B2
104,Toy_story(1995),A2/A2+
100,The_terminator(1984),B1


### Добавление слов из субтитров в таблицу фильмов

In [134]:
data['Subtitles'] = data.Movie.apply(prepare_subtitles, clean=True)

file datasets/subtitles/Glass Onion.srt not found
file datasets/subtitles/Matilda(2022).srt not found


> Для некоторых фильмов файл с субтитрами не найден. Однозначно определить, что это за фильм, и скачать для него субтитры не удалось.

### Удаление из таблицы фильмов без субтитров

In [135]:
data = data[data.Subtitles != 'no subtitles']

> Удалены фильмы, для которых не найден файл субтитров.

### Пример для ручной проверки

Очищенные субтитры

In [136]:
film_name = 'The.Man.Called.Flintstone.1966.1080p.WEB-DL.DD+2.0.H.264-DAWN'
prepare_subtitles(film_name, clean=True)

'who they call when the chips are startin fall just when the villains are winning the trouble beginning brew explosion the man called flintstone that who who always there there rescue some maiden fair man with instincts unerring full the old daring the man called flintstone that who thrives diet danger and stranger the unjust with romance who the man the man called flintstone rumbling screeches don let him get away don worry won hee hee hee hee hee hee hee hee hee angry muttering detour watch out snores angry muttering have parachute too good let follow him that our parachute hee hee hee hee hee hee hee hee hee some parachute shut you angry muttering crash have him trapped good let squish him ali hurry hurry must not escape won got him now panting giggles chuckles both laughing that the end rock slag nice work bobo thank you ali chuckles they think they finished off they don know how tough secret agents are this secret agent rock slag reporting chief boulder come chief this chief bould

Без очистки

In [137]:
prepare_subtitles(film_name, clean=False)

'ï»¿1 â\x99ª Who do they call â\x99ª â\x99ª When the chips are startin\' to fall â\x99ª â\x99ª Just when the villains are winning â\x99ª â\x99ª The trouble\'s beginning to brew â\x99ª [Explosion] â\x99ª The man called Flintstone â\x99ª â\x99ª That\'s who â\x99ª â\x99ª Who\'s always there â\x99ª â\x99ª There to rescue some maiden fair â\x99ª â\x99ª A man with instincts unerring â\x99ª â\x99ª Full of the old daring-do â\x99ª â\x99ª The man called Flintstone â\x99ª â\x99ª That\'s who â\x99ª â\x99ª He thrives on a diet of danger â\x99ª â\x99ª And he\'s no stranger â\x99ª â\x99ª To the unjust â\x99ª â\x99ª As he is with romance â\x99ª â\x99ª Who is the man â\x99ª â\x99ª The man called Flintstone â\x99ª [Rumbling] [Screeches] Don\'t let him get away. Don\'t worry. I won\'t. Ow! Hee hee! Hee hee hee! Hee hee! Hee hee-- [Angry muttering] Detour! Watch out! [Snores] [Angry muttering] We have parachute, too. Good. Let\'s follow him. That is our parachute? Hee hee! Hee hee hee! Hee hee! Hee hee! 

---

## Корректировка Level

Для фильмов с несколькими категориями нужно оставить одну (самую высокую).  
Заменить категории с плюсом на обычные, поскольку плюсовых категорий в CEFR нет.

In [138]:
data.Level = data.Level.apply(lambda Level: Level.replace('+', '')[-2:])

> Плюсы удалены.  
> При наличии нескольких категорий от экспертов для фильма установлена самая старшая категория (последние 2 символа).

---

## Save dataset for modeling

In [139]:
data.to_csv(f'datasets/EDA_movies_subtitles.csv', index=False)

> Предобработку субтитров можно/нужно добавить в пайплайн модели.  
> 
> Если не используются новые сгенерированные признаки, модель работает на новых данных и без предварительной обработки.

In [140]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 347 entries, 0 to 348
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Movie      347 non-null    object
 1   Level      347 non-null    object
 2   Subtitles  347 non-null    object
dtypes: object(3)
memory usage: 10.8+ KB


In [141]:
data.sample(5)

,Movie,Level,Subtitles
329,The.Umbrella.Academy.S03E05.720p.WEB.h264-KOGi,A2,great you again what owe the pleasure this time not talking huh well then listen not the mood fo...
175,Suits.S01E05.1080p.BluRay.AAC5.1.x265-DTG.02.EN,B2,ross triple double courtesy america favorite burger chain louis won the class action suit ten th...
304,The.Umbrella.Academy.S01E01.WEBRip.x264-ION10,A2,the hour the first day october this was unusual only the fact that none these women had been pre...
50,Lion(2016),B2,saroo saroo come come get come quickly hold properly saroo catch get down guddu the guards hey b...
35,Forrest_Gump(1994),B1,hello name forrest forrest gump you want chocolate could eat about million and half these mama a...


---

## Эксперименты

### New simple features

In [142]:
# # длина текста
# data['text_length'] = data.Subtitles.apply(lambda x: len(x))

# # длина текста без пробелов
# data['text_length_char'] = data.Subtitles.str.replace(' ', '').apply(lambda x: len(x))

# # средняя длина слова
# data['avg_word_length'] = data.text_length_char / (data.text_length - data.text_length_char)

> Эксперименты с моделью показали бесполезность этих новых признаков.

### Употребление слов в субтитрах из словаря Oxford

Будет исследовано распределение слов разных уровней в фильмах разных уровней.

Будет учтено как общее количество употреблений слова (даже если оно повторяется неоднократно), так и распределение уникальныхх слов (каждое слово будет учтено только один раз).

Идеальным резултатом будет выраженная зависимость между словами и фильмамит определенного уровня.

In [143]:
# файл содержит все слова из "American Oxford by CEFR level" с указанным уровнем

data_words = pd.read_csv(f'{DATASETS_PATH_LOCAL}EDA_word_level.csv')

In [144]:
# группировка слов по уровням и преобразование в списки

series_oxford = data_words.groupby('Level').word.agg(list)
series_oxford

Level
A1    [a, an, about, above, across, action, activity, actor, actress, add, advice, afraid, afternoon, ...
A2    [ability, able, accept, accident, according, achieve, active, actually, adult, advantage, advent...
B1    [absolutely, academic, access, account, achievement, act, ad, addition, administration, admire, ...
B2    [acknowledge, acquire, actual, adapt, additional, address, adopt, advance, affair, afterward, ag...
C1    [abolish, abortion, absence, absent, absurd, abuse, academy, accelerate, acceptance, accessible,...
Name: word, dtype: object

In [145]:
# преобразование текста с обработанными субтитрами в список, где каждый элемент – слово

data['subs_words'] = data.Subtitles.apply(lambda x: x.split())

In [146]:
def oxford_count(df, level):
    '''
    df: датафрейм для обработки
    level: уровень, соответствующий "American Oxford by CEFR level"
    
    Добавляет в датафрейм:
        поле вида 'level_sum' с количеством употреблений слов соответствующего уровня;
        поле вида 'level_unq' с количеством уникальных слов соответствующего уровня.
    '''
    
    subs = df['subs_words']
    oxford = series_oxford[level]
    
    df[f'{level}_sum'] = df['subs_words'].apply(lambda subs: sum(i in oxford for i in subs))
    df[f'{level}_unq'] = df['subs_words'].apply(lambda subs: sum(i in subs for i in oxford))
    
    return df

In [147]:
# список всех уровней CEFR, имеющихся в списке фильмов
level_list = sorted(list(data.Level.unique()))


# для каждого возможного уровня добавить в датафрейм:
# количество употреблений слов соответствующего уровня и
# количество уникальных слов соответствующего уровня

for level in tqdm(level_list):
    data = oxford_count(data, level)

100%|██████████| 5/5 [03:50<00:00, 46.19s/it]


In [148]:
# общее количество употреблений слов и уникальных слов всех уровней

data['TOTAL_sum'] = 0
data['TOTAL_unq'] = 0

for level in level_list:
    data['TOTAL_sum'] += data[f'{level}_sum']
    data['TOTAL_unq'] += data[f'{level}_unq']


# подсчет доли разных уровней для употреблений слов и уникальных слов

for level in level_list:
    data[f'{level}_sum_%'] = data[f'{level}_sum'] / data['TOTAL_sum']
    data[f'{level}_unq_%'] = data[f'{level}_unq'] / data['TOTAL_unq']

In [149]:
# удаление ненужных полей

data = data.drop(['subs_words','TOTAL_sum','TOTAL_unq'], axis=1)

for level in level_list:
    data = data.drop(f'{level}_sum', axis=1)     # количество употреблений слов определенного уровня
    data = data.drop(f'{level}_unq', axis=1)     # количество уникальных слов определенного уровня    

In [150]:
# сводная таблица для оценки доли употреблений слов и уникальных слов разных уровней
# в зависимости от уровня фильма, установленного экспертом

data.groupby('Level').mean()

,A1_sum_%,A1_unq_%,A2_sum_%,A2_unq_%,B1_sum_%,B1_unq_%,B2_sum_%,B2_unq_%,C1_sum_%,C1_unq_%
Level,,,,,,,,,,
A1,0.720,0.648,0.118,0.152,0.100,0.111,0.043,0.062,0.019,0.027
A2,0.626,0.442,0.143,0.226,0.117,0.141,0.079,0.141,0.035,0.050
B1,0.630,0.400,0.139,0.224,0.117,0.155,0.082,0.164,0.033,0.057
B2,0.640,0.395,0.126,0.223,0.120,0.156,0.080,0.165,0.034,0.061
C1,0.646,0.396,0.124,0.222,0.117,0.160,0.079,0.164,0.034,0.058


> Пояснение к названиям столбцов:  
> A1_sum_% – доля употребленных слов уровня A1 среди всех слов с известным уровнем.  
> A1_unq_% – доля уникальных слов уровня A1 среди всех слов с известным уровнем.  
> Аналогично и для других уровней.

> Если в полученной таблице изучить столбцы сверху вниз, то видно, что сколько либо заметно отличается только уровень A1, как для уникальных слов из словаря Oxford, так и для общего количества их употреблений.  
> Для остальных уровней доля слов разных уровней примерно одинакова.

> Эксперименты с моделированием подтвердили бесполезность этих новых признаков.

---

## Save dataset for modeling

In [151]:
# data.to_csv(f'datasets/EDA_movies_subtitles.csv', index=False)

> Сохранение датасета с новыми признаками

In [152]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 347 entries, 0 to 348
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Movie      347 non-null    object 
 1   Level      347 non-null    object 
 2   Subtitles  347 non-null    object 
 3   A1_sum_%   347 non-null    float64
 4   A1_unq_%   347 non-null    float64
 5   A2_sum_%   347 non-null    float64
 6   A2_unq_%   347 non-null    float64
 7   B1_sum_%   347 non-null    float64
 8   B1_unq_%   347 non-null    float64
 9   B2_sum_%   347 non-null    float64
 10  B2_unq_%   347 non-null    float64
 11  C1_sum_%   347 non-null    float64
 12  C1_unq_%   347 non-null    float64
dtypes: float64(10), object(3)
memory usage: 38.0+ KB


In [153]:
data.sample(5)

,Movie,Level,Subtitles,A1_sum_%,A1_unq_%,A2_sum_%,A2_unq_%,B1_sum_%,B1_unq_%,B2_sum_%,B2_unq_%,C1_sum_%,C1_unq_%
304,The.Umbrella.Academy.S01E01.WEBRip.x264-ION10,A2,the hour the first day october this was unusual only the fact that none these women had been pre...,0.626,0.436,0.131,0.212,0.121,0.146,0.082,0.148,0.040,0.058
220,Suits S04E13 EngSub,C1,and you got your name the wall now want the respect that comes with meaning want one harvey clie...,0.662,0.405,0.117,0.240,0.104,0.131,0.086,0.175,0.031,0.048
266,Gravity.Falls.S01E12.Summerween.720p.WEB-DL.AAC2.0.H264-Reaperza,A2,here are the summerween superstore wait summer what summerween the people this town love hallowe...,0.603,0.466,0.141,0.222,0.146,0.140,0.081,0.146,0.028,0.026
130,Spirit.Stallion.of.the.Cimarron.EN,B1,spirit stallion the cimarron the story want tell you can not found book they say the history the...,0.617,0.470,0.151,0.211,0.123,0.140,0.086,0.143,0.023,0.036
23,Clueless(1995),B1,okay you probably going this noxzema commercial what but seriously actually have way normal life...,0.633,0.380,0.135,0.201,0.119,0.166,0.084,0.185,0.029,0.068
